In [ ]:
from pyincore import IncoreClient, Dataset
from pyincore_data.censusutil import CensusUtil
from pyincore.analyses.housingrecoverysequential import HousingRecoverySequential
from pyincore.analyses.socialvulnerability import SocialVulnerability

import pandas as pd

In [ ]:
# Connect to IN-CORE service
client = IncoreClient()

### Chaining with social vulnerability analysis

#### prepare input datasets for social vulnerability analysis

In [ ]:
# Parameters
state = "texas"
county = "galveston"
year = 2020

In [ ]:
# get fips code to use fetch census data
fips = CensusUtil.get_fips_by_state_county(state=state, county=county)
state_code = fips[:2]
county_code = fips[2:]
fips

In [ ]:
navs = CensusUtil.national_ave_values(year=year)
national_vulnerability_feature_averages = Dataset.from_csv_data(navs, name="national_vulnerability_feature_averages.csv",
                                                                 data_type="incore:socialVulnerabilityFeatureAverages")

geo_type = "block%20group:*"
# geo_type = "tract:*"
social_vulnerability_dem_factors_df = CensusUtil.demographic_factors(state_code, county_code, year=year,
                                                                      geo_type=geo_type)

# Temp fix: remove bad data point
social_vulnerability_dem_factors_df = social_vulnerability_dem_factors_df.dropna()

social_vulnerability_dem_factors = Dataset.from_dataframe(social_vulnerability_dem_factors_df,
                                                           name="social_vunlnerability_dem_factors",
                                                           data_type="incore:socialVulnerabilityDemFactors")

#### run social vulnerability analysis

In [ ]:
# Configure pyincore access and the analysis with the information above

client = IncoreClient()
social_vulnerability = SocialVulnerability(client)

social_vulnerability.set_parameter("result_name", "social_vulnerabilty")
social_vulnerability.set_input_dataset("national_vulnerability_feature_averages", national_vulnerability_feature_averages)
social_vulnerability.set_input_dataset("social_vulnerability_demographic_factors", social_vulnerability_dem_factors)

In [ ]:
# Run social vulnerability damage analysis
result = social_vulnerability.run_analysis()

In [ ]:
# Retrieve result dataset
sv_result = social_vulnerability.get_output_dataset("sv_result")

# Convert dataset to Pandas DataFrame
df = sv_result.get_dataframe_from_csv()

# Display top 5 rows of output data
df.head()

### Household-level housing serial recovery for Galveston testbed.

In [ ]:
# Population dislocation result aggregated to the block group level.
population_dislocation = "60f5ea5d544e944c3cec66dd" # Galveston testbed
# Transition probability matrix per social vulnerability level.
transition_probability_matrix = "60f5e2ae544e944c3cec0794"
# Initial mass probability function for stage 0.
initial_probability_vector = "60f5e918544e944c3cec668b"

In [ ]:
# Create housing recovery instance
housing_recovery = HousingRecoverySequential(client)

# Load input datasets
housing_recovery.load_remote_input_dataset("population_dislocation_block", population_dislocation)
housing_recovery.load_remote_input_dataset("tpm", transition_probability_matrix)
housing_recovery.load_remote_input_dataset("initial_stage_probabilities", initial_probability_vector)

In [ ]:
# Chain with SV output
housing_recovery.set_input_dataset('sv_result', sv_result)

In [ ]:
# Initial value to seed the random number generator to ensure replication
seed = 1234
# A size of the analysis time step.
t_delta = 1.0
# Total duration.
t_final = 90.0

In [ ]:
# Specify the result name
result_name = "housing_recovery_result"

# Set analysis parameters
housing_recovery.set_parameter("result_name", result_name)
housing_recovery.set_parameter("seed", seed)
housing_recovery.set_parameter("t_delta", t_delta)
housing_recovery.set_parameter("t_final", t_final)

In [ ]:
# Run the analysis
housing_recovery.run()

In [ ]:
# Retrieve result dataset
result = housing_recovery.get_output_dataset("ds_result")

# Convert dataset to Pandas DataFrame
df = result.get_dataframe_from_csv()

# Display top 5 rows of output data
df.head()